<a href="https://colab.research.google.com/github/jordannym/ETL_Python/blob/main/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio de Código (ETL com Python)

**Contexto:** Você é um cientista de dados em uma empresa de serviço de streaming e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é utilizar os gêneros de filmes favoritos de cada cliente, para recomendar filmes.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  id
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://retoolapi.dev/TK0gIL/usuarios/{id}` (API dos usuários 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar o catálogo de filmes em formato CSV ('catalogo.csv') da plataforma para retornar nomes de filmes correspondentes ao gênero indicado.
4. Uma vez que a recomendação para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "filmes" de cada usuário usando o endpoint `PUT https://api-generator.retool.com/TK0gIL/usuarios/{id}`.





#Extract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [8]:
import pandas as pd

# Read CSV
df = pd.read_csv('/DCP2023.csv')
user_ids = df['id'].tolist()

# Visualize user IDs
print(user_ids)

[1, 2, 3, 4, 5]


In [39]:
import requests
import json

users2023_api_url = "https://retoolapi.dev/oXNTDh"

# Recuperate users (Id)
def get_user(id):
  response = requests.get(f'{users2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

# Visualize users information from API
users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))



[
  {
    "id": 1,
    "email": "mturpin0@issuu.com",
    "gender": "Male",
    "last_name": "Turpin",
    "first_name": "Malcolm",
    "favorite_movie_genres": "Action"
  },
  {
    "id": 2,
    "email": "fizzett1@over-blog.com",
    "gender": "Female",
    "last_name": "Izzett",
    "first_name": "Farrand",
    "favorite_movie_genres": "Drama"
  },
  {
    "id": 3,
    "email": "jorbon2@nifty.com",
    "gender": "Female",
    "last_name": "Orbon",
    "first_name": "Jenda",
    "favorite_movie_genres": "Documentary"
  },
  {
    "id": 4,
    "email": "egeraudel3@washingtonpost.com",
    "gender": "Male",
    "last_name": "Geraudel",
    "first_name": "Enoch",
    "favorite_movie_genres": "Comedy"
  },
  {
    "id": 5,
    "email": "obaldree4@jiathis.com",
    "gender": "Male",
    "last_name": "Baldree",
    "first_name": "Osgood",
    "favorite_movie_genres": "Animation"
  }
]


#Transform

 Utilize o catálogo de filmes em formato CSV ('catalogo.csv') da plataforma para retornar nomes de filmes correspondentes ao gênero favorito de cada usuário.



In [62]:
# Data from streaming catalog
catalog=pd.read_csv('/catalog2023.csv')
catalog

,id,movie,movie_genre
0,1,Scatter My Ashes at Bergdorf's,Documentary
1,2,Edie & Pen,Comedy
2,3,Quill,Drama
3,4,"Brothers Rico, The",Crime|Drama|Film-Noir|Thriller
4,5,Trespass,Crime|Drama|Thriller
...,...,...,...
995,996,"Amish Murder, An",Action|Crime|Drama
996,997,Asterix vs. Caesar (Astérix et la surprise de ...,Adventure|Animation|Children|Comedy
997,998,She Gods of Shark Reef,Adventure
998,999,Pusher III: I'm the Angel of Death,Action|Comedy|Drama|Horror|Thriller


In [60]:
# Data from users API

df = pd.DataFrame(users)
df.to_csv('users_info.csv', index=False, header=True)

users_info=pd.read_csv('users_info.csv')
users_info


,id,email,gender,last_name,first_name,favorite_movie_genres
0,1,mturpin0@issuu.com,Male,Turpin,Malcolm,Action
1,2,fizzett1@over-blog.com,Female,Izzett,Farrand,Drama
2,3,jorbon2@nifty.com,Female,Orbon,Jenda,Documentary
3,4,egeraudel3@washingtonpost.com,Male,Geraudel,Enoch,Comedy
4,5,obaldree4@jiathis.com,Male,Baldree,Osgood,Animation


Usar um LeftJoin para recomendar filmes do catálogo correspondentes ao gênero favorito de cada usuário.

In [71]:
merged_data = users_info.merge(catalog, left_on="favorite_movie_genres", right_on="movie_genre", how='left')
merged_data.head()

,id_x,email,gender,last_name,first_name,favorite_movie_genres,id_y,movie,movie_genre
0,1,mturpin0@issuu.com,Male,Turpin,Malcolm,Action,214,"Hunted, The",Action
1,1,mturpin0@issuu.com,Male,Turpin,Malcolm,Action,366,Super Bitch,Action
2,1,mturpin0@issuu.com,Male,Turpin,Malcolm,Action,367,Knock Off,Action
3,1,mturpin0@issuu.com,Male,Turpin,Malcolm,Action,580,"Shaolin Temple, The (Shao Lin Si)",Action
4,1,mturpin0@issuu.com,Male,Turpin,Malcolm,Action,641,American Kickboxer 2 (American Kickboxer 2: To...,Action


#Load

Atualize a coluna de "filmes recomendados" de cada usuário na API

In [75]:
def update_user(user):
  response = requests.put(f"{users2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['first_name']} updated? {success}!")

User Malcolm updated? True!
User Farrand updated? True!
User Jenda updated? True!
User Enoch updated? True!
User Osgood updated? True!
